In [17]:
from sklearn.model_selection import KFold
import pandas as pd

In [18]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [19]:
sweep_config = {
    'method': 'random'
}

In [20]:
metric = {
    'name': 'Grand Mean',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [21]:
parameters_dict = {
    'dropout': {
          'values': [0.71]
        },

    'learning_rate': {
        'values': [0.0005]
        },

    'batch_size': {
        'values': [4096]
        },

    'data_augmentation_multiple': {
        'values': [5]
        }
}

In [22]:
sweep_config['parameters'] = parameters_dict

In [23]:
sweep_id = wandb.sweep(sweep_config, project="sub_loc_train")

Create sweep with ID: 3fo2ht7f
Sweep URL: https://wandb.ai/imucs/sub_loc_train/sweeps/3fo2ht7f


In [24]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Grand Mean'},
 'parameters': {'batch_size': {'values': [4096]},
                'data_augmentation_multiple': {'values': [5]},
                'dropout': {'values': [0.71]},
                'learning_rate': {'values': [0.0005]}}}


In [25]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_right.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [26]:
from dataAug.tools import MLDA

In [27]:
from dataAug.all_tools import dataAugSMOTE

In [28]:

smote_multiple = {}

In [29]:
import time
from classify.targeTools import testThresholdFive, Accuracy
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from classify.Classify_adjust import ModelClassify
def train_and_val(config=None):
    with wandb.init(config=config):
        config = wandb.config
        kf = KFold(n_splits=10, shuffle=True)
        model_discord = []

        multi_label_samples = labels_pd[(labels_pd.sum(axis=1) >= 2)]
        multi_label_indices = multi_label_samples.index
        multi_features_samples = feature_pd.loc[multi_label_indices]
        ML_G_X, ML_G_y = MLDA(multi_features_samples, multi_label_samples, config.data_augmentation_multiple)

        G_feature, G_label = dataAugSMOTE(feature_pd, labels_pd, config.data_augmentation_multiple, 1424)
        G_feature = pd.concat([G_feature, ML_G_X], axis=0)
        G_label = pd.concat([G_label, ML_G_y], axis=0)
        feature_all = pd.concat([feature_pd, G_feature], axis=0)
        label_all = pd.concat([labels_pd, G_label], axis=0)
        print(feature_all.shape)
        print(label_all.shape)
        for train_index, test_index in kf.split(feature_all):
            train_data = feature_all.iloc[train_index]
            train_label = label_all.iloc[train_index]

            test_data = feature_all.iloc[test_index]
            test_label = label_all.iloc[test_index]

            datasetTrain = TensorDataset(torch.tensor(train_data.values), torch.tensor(train_label.values))
            batch_size = config.batch_size
            dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

            datasetTest = TensorDataset(torch.tensor(test_data.values), torch.tensor(test_label.values))

            batch_size = len(datasetTest)
            dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

            model = ModelClassify(drop_rate=config.dropout, num_class=5, feature_num=1424)
            criterion = torch.nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
            import warnings
            warnings.filterwarnings("ignore")
            GMList = {}
            epochs = 100
            for epoch in range(epochs):
                model.train()
                total_loss = 0.0
                for idx, data in enumerate(dataloaderTrain, 0):
                    inputs, labels = data
                    labels = labels.float()
                    inputs = inputs.float()
                    out = model(inputs)
                    loss = criterion(out, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                avg_loss = total_loss / len(dataloaderTrain)
                threshold = 0.5
                labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
                print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
            GMScore1 = testThresholdFive(epoch, model, dataloaderTest, class_num=5)
            GMList[epoch] = GMScore1
            t = time.time()
            best_key = max(GMList, key=GMList.get)
            best_value = GMList[best_key]

            best_value = [x.item() if isinstance(x, torch.Tensor) else x for x in best_value]
            model_discord.append(best_value)
        model_discord_column_means = [sum(col) / len(col) for col in zip(*model_discord)]
        smote_multiple[int(t)] = model_discord_column_means
        wandb.log({"Grand Mean": model_discord_column_means[0]})

In [30]:
wandb.agent(sweep_id, train_and_val, count=30)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: Ctrl + C detected. Stopping sweep.


In [15]:
smote_multiple

{}

In [16]:
import json
file = open("/home/kongge/projects/new_protT5/data/dictionary_data_DADL.json", "w")
json.dump(smote_multiple, file)
file.close()